In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
tf.logging.set_verbosity(tf.logging.INFO)

# 1. Preprocessing Data

In [4]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

In [6]:
training_set = pd.read_csv("./datasets/boston/boston_train.csv", skipinitialspace=True, skiprows=1, names=COLUMNS)
test_set = pd.read_csv("./datasets/boston/boston_test.csv", skipinitialspace=True, skiprows=1, names=COLUMNS)
predict_set = pd.read_csv("./datasets/boston/boston_predict.csv", skipinitialspace=True, skiprows=1, names=COLUMNS)
training_set.head()

,crim,zn,indus,nox,rm,age,dis,tax,ptratio,medv
0,2.30040,0.0,19.58,0.605,6.319,96.1,2.1000,403,14.7,23.8
1,13.35980,0.0,18.10,0.693,5.887,94.7,1.7821,666,20.2,12.7
2,0.12744,0.0,6.91,0.448,6.770,2.9,5.7209,233,17.9,26.6
3,0.15876,0.0,10.81,0.413,5.961,17.5,5.2873,305,19.2,21.7
4,0.03768,80.0,1.52,0.404,7.274,38.3,7.3090,329,12.6,34.6


In [7]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

# 2. Train Data

In [8]:
dnn_regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols, 
                                          hidden_units=[10, 10], model_dir="/tmp/tensorflow/boston")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x181f1b4910>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tensorflow/boston', '_save_summary_steps': 100}


In [14]:
def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y = pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)

In [15]:
dnn_regressor.train(input_fn=get_input_fn(training_set), steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tensorflow/boston/model.ckpt.
INFO:tensorflow:loss = 142116.0, step = 1
INFO:tensorflow:global_step/sec: 248.102
INFO:tensorflow:loss = 15178.8, step = 101 (0.413 sec)
INFO:tensorflow:global_step/sec: 305.14
INFO:tensorflow:loss = 12334.7, step = 201 (0.324 sec)
INFO:tensorflow:global_step/sec: 311.431
INFO:tensorflow:loss = 10152.7, step = 301 (0.323 sec)
INFO:tensorflow:global_step/sec: 266.91
INFO:tensorflow:loss = 12844.6, step = 401 (0.376 sec)
INFO:tensorflow:global_step/sec: 281.385
INFO:tensorflow:loss = 9470.42, step = 501 (0.356 sec)
INFO:tensorflow:global_step/sec: 254.452
INFO:tensorflow:loss = 11679.1, step = 601 (0.394 sec)
INFO:tensorflow:global_step/sec: 279.762
INFO:tensorflow:loss = 7772.12, step = 701 (0.353 sec)
INFO:tensorflow:global_step/sec: 309.28
INFO:tensorflow:loss = 12427.7, step = 801 (0.324 sec)
INFO:tensorflow:global_step/sec: 178.11
INFO:tensorflow:loss = 11204

# 3. Evalute the Model

In [16]:
ev = dnn_regressor.evaluate(
    input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))

INFO:tensorflow:Starting evaluation at 2017-12-09-14:00:12
INFO:tensorflow:Restoring parameters from /tmp/tensorflow/boston/model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2017-12-09-14:00:13
INFO:tensorflow:Saving dict for global step 5000: average_loss = 23.7327, global_step = 5000, loss = 2373.27


# 4. Make Predictions

In [21]:
predictions = dnn_regressor.predict(input_fn=get_input_fn(predict_set, num_epochs=1, shuffle=False))
predictions = list(predictions)

INFO:tensorflow:Restoring parameters from /tmp/tensorflow/boston/model.ckpt-5000


In [22]:
predictions

[{'predictions': array([ 37.74973679], dtype=float32)},
 {'predictions': array([ 21.14399529], dtype=float32)},
 {'predictions': array([ 24.50967026], dtype=float32)},
 {'predictions': array([ 34.70974731], dtype=float32)},
 {'predictions': array([ 16.32670403], dtype=float32)},
 {'predictions': array([ 21.40563965], dtype=float32)}]